In [1]:
import numpy as np
import pandas as pd
from scipy.stats import nbinom


In [14]:
import numpy as np
import pandas as pd
from scipy.stats import nbinom, zipf

# set random seed for reproducibility
np.random.seed(123)

# generate dummy data using Zipf distribution
n_items = 1000
freq_rank = np.arange(1, n_items + 1)
freq = 1 / freq_rank
freq = freq / freq.sum()  # normalize frequencies to sum to 1
item_ids = np.arange(1, n_items + 1)
timestamp = np.random.randint(0, 1000000, size=n_items)
data = pd.DataFrame({'user_id': np.repeat(1, n_items), 'item_id': item_ids, 'timestamp': timestamp})

# fit discrete negative binomial distribution
params = nbinom.fit(data.groupby('item_id').size())
dist = nbinom(*params)

# expand/extrapolate distribution
max_freq = data.groupby('item_id').size().max()
freq_range = np.arange(max_freq + 1)
probs = dist.pmf(freq_range)

# sample 100 items from extrapolated distribution
sampled_freq = np.random.choice(freq_range, size=100, p=probs)
sampled_items = []
for i, f in enumerate(sampled_freq):
    item_id = data.loc[data['item_id'].value_counts() == f, 'item_id'].iloc[0]
    sampled_items.append((i, item_id))

# create interpolation function
interp_fn = np.interp(np.linspace(0, 1, len(probs)), probs, freq_range)

# usage example
# get index position of item_id = 123
idx_pos = data.loc[data['item_id'] == 123, 'item_id'].index[0]
# get probability of frequency 3
prob = dist.pmf(3)
# get interpolated index position for probability of frequency 3
interp_pos = interp_fn[prob]

AttributeError: 'nbinom_gen' object has no attribute 'fit'

In [7]:
def create_index_df_from_column(argument_df, columnname):
    # create an index for the training item_ids, because we need this when training the model
    # this should be part of the train and serve code of an algorithm
    # usage:
    #     create_index_df_from_column(df, 'ItemId')
    # returns a two column dataframe:
    #   ItemId  idx
    #   9200000105095118    0
    #   9200000041902190    1
    #   9200000095595490    2
    token_id = {t: i for i, t in enumerate(argument_df[columnname].unique())}
    df_index = pd.DataFrame.from_dict(token_id, orient="index").reset_index()
    df_index.rename(columns={"index": columnname, 0: "idx"}, inplace=True)
    return df_index

In [10]:
item_pop_df = df.groupby('ItemId').size()
item_pops_df = item_pop_df.to_frame('freq').reset_index()
item_pops_df.sort_values(by=['freq'], inplace=True)
item_idx = create_index_df_from_column(item_pops_df, 'ItemId')
dfs = item_pops_df.merge(item_idx, on='ItemId')
dfs

,ItemId,freq,idx
0,9200000092996478,1,0
1,9200000066211433,1,1
2,9200000123889779,1,2
3,9200000066211493,1,3
4,9200000066211497,1,4
...,...,...,...
338412,9200000110533529,2073,338412
338413,9200000056292168,2177,338413
338414,9200000026848187,2186,338414
338415,9200000047735877,2224,338415


In [13]:
# load dataset
dfs['index_pos'] = range(len(dfs))

# fit discrete negative binomial distribution
params = nbinom.fit(dfs['freq'])
dist = nbinom(*params)

# expand/extrapolate distribution
max_freq = dfs['freq'].max()
freq_range = np.arange(max_freq + 1)
probs = dist.pmf(freq_range)

# sample 100 items from extrapolated distribution
sampled_freq = np.random.choice(freq_range, size=100, p=probs)
sampled_items = []
for i, f in enumerate(sampled_freq):
    item_id = freq.loc[dfs['freq'] == f, 'item_id'].iloc[0]
    sampled_items.append((i, item_id))

# create interpolation function
interp_fn = np.interp(np.linspace(0, 1, len(probs)), probs, freq_range)

# usage example
# get index position of item_id = 123
idx_pos = dfs.loc[dfs['item_id'] == 123, 'index_pos'].iloc[0]
# get probability of frequency 3
prob = dist.pmf(3)
# get interpolated index position for probability of frequency 3
interp_pos = interp_fn[prob]

AttributeError: 'nbinom_gen' object has no attribute 'fit'